### Bitcoin block sampler
[Bitcoin core RPC API](https://en.bitcoin.it/wiki/Original_Bitcoin_client/API_calls_list)

[Bitcoin core RPC Doc](https://bitcoin.org/en/developer-reference#remote-procedure-calls-rpcs)

#### Operation ordering


In [ ]:
# For secrets
from secret import rpc_user, rpc_password

In [ ]:
## Check the bitcoind running
import os

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind PID: {f.read()}')
else:
    raise SystemExit('Bitcoind is running!')

datadir = os.path.abspath(os.path.expanduser('~/.bitcoin/regtest'))
pid_path = os.path.join(datadir, 'bitcoind.pid')
if os.path.exists(pid_path):
    with open(pid_path, 'r') as f:
        print(f'Bitcoind -chain=regtest PID: {f.read()}')
else:
    raise SystemExit('Bitcoind -chain=regtest is running!')

In [ ]:
## Some function for convenient
import datetime

tz_seoul = datetime.timezone(datetime.timedelta(hours=9))
tz_utc = datetime.timezone(datetime.timedelta())

def get_time(timestamp):
    return datetime.datetime.fromtimestamp(timestamp, tz=tz_seoul)

In [ ]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException
rpc_ip = '127.0.0.1'
rpc_port = '8332'
rpc_port_regtest = '18443'
timeout = 300

rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', timeout=timeout)
best_block_hash = rpc_connection.getbestblockhash()
best_block = rpc_connection.getblock(best_block_hash)
print(f'Best Main Block Heights: {best_block["height"]}, Time: {get_time(best_block["time"]).isoformat()}')

### Flow
1. Find address set by address apearance type
  - Unlinked addresses
  - Linked addresses

In [ ]:
def get_balance(rpc_connection, v):
    tx = rpc_connection.getrawtransaction(v['txid'], 1)
    block = rpc_connection.getblock(tx['blockhash'])
    addr = tx['vout'][v['vout']]['scriptPubKey']['addresses'][0]
    height = block['height']
    value = tx['vout'][v['vout']]['value']
    return addr, height, value

In [ ]:
# Input block
sheight = int(input('Start block height: '))
eheight = int(input('End block height: '))

In [ ]:
%%time
import time

rpc_connection = AuthServiceProxy(f'http://{rpc_user}:{rpc_password}@{rpc_ip}:{rpc_port}', 
                                  timeout=timeout)

base = list()

time1 = time.time()
for i in range(sheight, eheight+1):
    tbase = list()
    block_hash = rpc_connection.getblockhash(i)
    block = rpc_connection.getblock(block_hash) # json
    for txhash in block['tx'][1:]:
        print(f'{i} block processing: {txhash}', end='\r')
        tx = rpc_connection.getrawtransaction(txhash, 1) # json => dict 연산 object 대상
        for v in tx['vin']:
            addr, height, value = get_balance(rpc_connection, v)
            if not (sheight <= height <= eheight):
                tbase.append((addr, value))
    base.extend(tbase)
    time2 = time.time()
    print(f'[{round(time2-time1)}] {len(base)} from Reading {i} / {eheight} ({(i-sheight)/(eheight-sheight)*100}%)')
    time1 = time2

In [ ]:
# 578755 block processing: 3141d7e07774f6e9d2d9266bd2070d996b0ae946b464acd5dbe47007da4faf31
# https://github.com/bitcoinjs/bitcoinjs-lib/issues/1000